In [15]:
import fastf1 as ff1
from fastf1 import plotting
from matplotlib import pyplot as plt
from matplotlib.collections import LineCollection
from matplotlib import cm
import numpy as np
import pandas as pd

ff1.Cache.enable_cache("cache")
plotting.setup_mpl()

In [16]:
q = ff1.get_session(2022, "Bahrain", "Q")
q.load(telemetry=True)

core           INFO 	Loading data for Bahrain Grand Prix - Qualifying [v2.2.1]
api            INFO 	Using cached data for driver_info
api            INFO 	Using cached data for timing_data
api            INFO 	Using cached data for timing_app_data
core           INFO 	Processing timing data...
api            INFO 	Using cached data for session_status_data
api            INFO 	Using cached data for track_status_data
api            INFO 	Using cached data for car_data
api            INFO 	Using cached data for position_data
api            INFO 	Using cached data for weather_data
core           INFO 	Finished loading data for 20 drivers: ['16', '1', '55', '11', '44', '77', '20', '14', '63', '10', '31', '47', '4', '23', '24', '22', '27', '3', '18', '6']


In [17]:
lap_russell = q.laps.pick_driver("RUS").pick_fastest().get_car_data().add_distance()
lap_perez = q.laps.pick_driver("PER").pick_fastest().get_car_data().add_distance()

lap_russell["Driver"] = "RUS"
lap_perez["Driver"] = "PER"

In [18]:
telemetry = pd.concat([lap_russell, lap_perez], axis=0)

In [19]:
telemetry

,Date,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Time,SessionTime,Distance,Driver
0,2022-03-19 15:38:59.208,10673,289,8,100,False,12,car,0 days 00:00:00.004000,0 days 00:53:58.279000,0.321111,RUS
1,2022-03-19 15:38:59.488,10764,292,8,100,False,12,car,0 days 00:00:00.284000,0 days 00:53:58.559000,23.032222,RUS
2,2022-03-19 15:38:59.688,10770,293,8,100,False,12,car,0 days 00:00:00.484000,0 days 00:53:58.759000,39.310000,RUS
3,2022-03-19 15:39:00.128,10912,296,8,100,False,12,car,0 days 00:00:00.924000,0 days 00:53:59.199000,75.487778,RUS
4,2022-03-19 15:39:00.408,10992,298,8,100,False,12,car,0 days 00:00:01.204000,0 days 00:53:59.479000,98.665556,RUS
...,...,...,...,...,...,...,...,...,...,...,...,...
332,2022-03-19 16:00:21.492,11205,280,7,100,False,12,car,0 days 00:01:29.891000,0 days 01:15:20.563000,5281.917222,PER
333,2022-03-19 16:00:21.731,11304,284,7,100,False,12,car,0 days 00:01:30.130000,0 days 01:15:20.802000,5300.771667,PER
334,2022-03-19 16:00:22.051,11336,288,7,100,False,12,car,0 days 00:01:30.450000,0 days 01:15:21.122000,5326.371667,PER
335,2022-03-19 16:00:22.211,11440,289,7,100,False,12,car,0 days 00:01:30.610000,0 days 01:15:21.282000,5339.216111,PER


In [20]:
num_minisectors = 30

total_distance = max(telemetry["Distance"])

minisector_length = total_distance / num_minisectors

In [21]:
minisectors = [0]
for i in range(0, (num_minisectors - 1)):
    minisectors.append(minisector_length * (i + 1))

In [22]:
telemetry["Minisector"] = telemetry["Distance"].apply(
    lambda dist: (int((dist // minisector_length) + 1))
)

average_speed = (
    telemetry.groupby(["Minisector", "Driver"])["Speed"].mean().reset_index()
)

In [23]:
telemetry

,Date,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Time,SessionTime,Distance,Driver,Minisector
0,2022-03-19 15:38:59.208,10673,289,8,100,False,12,car,0 days 00:00:00.004000,0 days 00:53:58.279000,0.321111,RUS,1
1,2022-03-19 15:38:59.488,10764,292,8,100,False,12,car,0 days 00:00:00.284000,0 days 00:53:58.559000,23.032222,RUS,1
2,2022-03-19 15:38:59.688,10770,293,8,100,False,12,car,0 days 00:00:00.484000,0 days 00:53:58.759000,39.310000,RUS,1
3,2022-03-19 15:39:00.128,10912,296,8,100,False,12,car,0 days 00:00:00.924000,0 days 00:53:59.199000,75.487778,RUS,1
4,2022-03-19 15:39:00.408,10992,298,8,100,False,12,car,0 days 00:00:01.204000,0 days 00:53:59.479000,98.665556,RUS,1
...,...,...,...,...,...,...,...,...,...,...,...,...,...
332,2022-03-19 16:00:21.492,11205,280,7,100,False,12,car,0 days 00:01:29.891000,0 days 01:15:20.563000,5281.917222,PER,30
333,2022-03-19 16:00:21.731,11304,284,7,100,False,12,car,0 days 00:01:30.130000,0 days 01:15:20.802000,5300.771667,PER,30
334,2022-03-19 16:00:22.051,11336,288,7,100,False,12,car,0 days 00:01:30.450000,0 days 01:15:21.122000,5326.371667,PER,30
335,2022-03-19 16:00:22.211,11440,289,7,100,False,12,car,0 days 00:01:30.610000,0 days 01:15:21.282000,5339.216111,PER,30


In [24]:
fastest_driver = average_speed.loc[
    average_speed.groupby(["Minisector"])["Speed"].idxmax()
]

fastest_driver = fastest_driver[["Minisector", "Driver"]].rename(
    columns={"Driver": "Fastest_driver"}
)

telemetry = telemetry.merge(fastest_driver, on=["Minisector"])
telemetry = telemetry.sort_values(by=["Distance"])

telemetry.loc[telemetry["Fastest_driver"] == "GAS", "Fastest_driver_int"] = 1
telemetry.loc[telemetry["Fastest_driver"] == "BOT", "Fastest_driver_int"] = 2

telemetry

,Date,RPM,Speed,nGear,Throttle,Brake,DRS,Source,Time,SessionTime,Distance,Driver,Minisector,Fastest_driver,Fastest_driver_int
0,2022-03-19 15:38:59.208,10673,289,8,100,False,12,car,0 days 00:00:00.004000,0 days 00:53:58.279000,0.321111,RUS,1,PER,NaN
1,2022-03-19 15:38:59.488,10764,292,8,100,False,12,car,0 days 00:00:00.284000,0 days 00:53:58.559000,23.032222,RUS,1,PER,NaN
8,2022-03-19 15:58:51.891,10456,296,8,100,False,12,car,0 days 00:00:00.290000,0 days 01:13:50.962000,23.844444,PER,1,PER,NaN
9,2022-03-19 15:58:52.051,10705,296,8,100,False,12,car,0 days 00:00:00.450000,0 days 01:13:51.122000,37.000000,PER,1,PER,NaN
2,2022-03-19 15:38:59.688,10770,293,8,100,False,12,car,0 days 00:00:00.484000,0 days 00:53:58.759000,39.310000,RUS,1,PER,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
665,2022-03-19 15:40:29.968,11622,285,7,100,False,12,car,0 days 00:01:30.764000,0 days 00:55:29.039000,5324.998056,RUS,30,PER,NaN
675,2022-03-19 16:00:22.051,11336,288,7,100,False,12,car,0 days 00:01:30.450000,0 days 01:15:21.122000,5326.371667,PER,30,PER,NaN
676,2022-03-19 16:00:22.211,11440,289,7,100,False,12,car,0 days 00:01:30.610000,0 days 01:15:21.282000,5339.216111,PER,30,PER,NaN
677,2022-03-19 16:00:22.371,11536,290,7,100,False,12,car,0 days 00:01:30.770000,0 days 01:15:21.442000,5352.105000,PER,30,PER,NaN


In [25]:
x_bot = np.array(telemetry[telemetry.Driver == "BOT"]["Speed"].values)
y_bot = np.array(telemetry[telemetry.Driver == "BOT"]["Time"].values)

x_gas = np.array(telemetry[telemetry.Driver == "GAS"]["Speed"].values)
y_gas = np.array(telemetry[telemetry.Driver == "GAS"]["Time"].values)

In [26]:
import plotly.express as px

fig = px.line(telemetry, "Distance", "Speed", color="Driver")
fig.show()

In [27]:
fig = px.line(telemetry, "Time", "Throttle", color="Driver")
fig.show()

In [28]:
telemetry.groupby("Driver").Time.max()

Driver
PER   0 days 00:01:30.770000
RUS   0 days 00:01:31.123000
Name: Time, dtype: timedelta64[ns]